In [39]:
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus as wkc


In [40]:
wikidspath = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
sentences = wkc(wikidspath).get_texts()

In [54]:
nltk.download('punkt')
nltk.download('reuters')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [41]:
rs  = reuters.sents()

In [42]:
print(rs)

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [47]:
def wordsProbability(sentenceModel):
  for nextWord in sentenceModel:
    nextWords = sentenceModel[nextWord]
    totalWordCount = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord] /= totalWordCount

def singleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word] /= wordCount

def convertToLower(s):
  if type(s)==str:
    return s.lower()
  return s

In [29]:
sentenceModel4 = defaultdict(lambda: set())
sentenceModel5 = defaultdict(lambda: set())

def calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,sentences):
  wordCount = 0
  for sentence in sentences:
    for word in sentence:
      wordCount += 1
      sentenceModel1[word] += 1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      nextWord = convertToLower(nextWord)
      sentenceModel2[nextWord][previousWord1] += 1 
      sentenceModel3[nextWord][previousWord2] += 1
      sentenceModel4[previousWord1].add(nextWord)
      sentenceModel5[previousWord2].add(nextWord) 

  return wordCount

In [30]:
sentenceModel1 = defaultdict(lambda:0)
sentenceModel2 = defaultdict(lambda: defaultdict(lambda:0))
sentenceModel3 = defaultdict(lambda: defaultdict(lambda:0))

WikiWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,WikiSentences)
print(WikiWordCount)

452944


In [21]:
ReutersWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,ReutersSentences)
print(ReutersWordCount)

1720917


In [31]:
calculateWordsProbabilities(sentenceModel2)
calculateWordsProbabilities(sentenceModel3)

totalWord = WikiWordCount + ReutersWordCount
calculateSingleWordProbability(sentenceModel1,totalWord)

In [57]:
ProbabilityWordsList = [] 

def wordSuggestion(previousWord2,previousWord1): 
  for nextWord in sentenceModel4[previousWord1] & sentenceModel5[previousWord2]:
    naiveBayesTrigramValue = sentenceModel1[nextWord]*sentenceModel2[nextWord][previousWord1]*sentenceModel3[nextWord][previousWord2]
    ProbabilityWordsList.append((nextWord,naiveBayesTrigramValue))

In [67]:
text = input("Text: ")
ProbabilityWordsList = [] 
text = text.split(" ")
wordSuggestion(text[0],text[1])
ProbabilityWordsList.sort(key=lambda o:o[1],reverse=True)
print(*ProbabilityWordsList[:10])

Text: he needs
('to', 1.559359364961502e-08) ('that', 7.821940408821843e-09) ('and', 3.094102899178931e-09) ('it', 2.8855892064629994e-09) ('the', 2.230569445442022e-09) ('in', 1.986452993509535e-09) ('this', 1.306849467794441e-09) ('more', 1.0849316336406677e-09) ('of', 8.868631958384381e-10)
